In [3]:
from google.colab import drive
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path to your ZIP file in Google Drive
zip_path = '/content/drive/MyDrive/dog-breed-identification.zip'

# Destination to extract the ZIP
extract_path = '/content/dog-breed-identification'

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Set the new dataset path for use in the rest of the code
dataset_path = extract_path


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: 

In [4]:
import torch
import os
from PIL import Image
from pathlib import Path
import pandas as pd
import torchvision
from sklearn.model_selection import train_test_split

In [3]:
labels=pd.read_csv('/content/dog-breed-identification/labels.csv')
train,valid=train_test_split(labels,train_size=0.8,shuffle=True,stratify=labels['breed'],random_state=42)

In [4]:
train,train_labels=train['id'].reset_index(drop=True),train['breed'].reset_index(drop=True)
val,val_labels=valid['id'].reset_index(drop=True),valid['breed'].reset_index(drop=True)

In [5]:
'''Encoding labels'''

breeds=dict()
breed_count=1

for breed in labels['breed'].value_counts().index:

    breeds[breed]=breed_count-1
    breed_count+=1

val_labels_torch=torch.zeros(len(val_labels),1)
train_labels_torch=torch.zeros(len(train_labels),1)

for index in val_labels.index:
    val_labels_torch[index]=breeds[val_labels.iloc[index]]

for index in train_labels.index:
    train_labels_torch[index]=breeds[train_labels.iloc[index]]

val_labels_torch = val_labels_torch.long()
train_labels_torch = train_labels_torch.long()


val_labels_torch[91]
# val_labels=torch.tensor(val_labels.values)
# train_labels=torch.tensor(train_labels.values)

tensor([21])

In [6]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(10),
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    torchvision.transforms.ToTensor()
])


def create_dataset(series, directory_path, transform):
    tensors = []
    for i in range(len(series)):
        if(series[i].endswith('.jpg')):
            img_file=series[i]
        else:
            img_file = series[i] + '.jpg'
        img_path = os.path.join(directory_path, img_file)
        img = Image.open(img_path).convert('RGB')  # Ensure RGB
        img_t = transform(img)
        tensors.append(img_t)
    return torch.stack(tensors, dim=0)


In [7]:
directory_path=Path('/content/dog-breed-identification/train')
train_dataset=create_dataset(train,directory_path,transform)

In [8]:


# For validation: no augmentation
val_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor()
])
val_dataset = create_dataset(val, directory_path, transform=val_transform)


In [9]:
from torch.utils.data import TensorDataset

train_dataset_torch=TensorDataset(train_dataset,train_labels_torch)
val_dataset_torch=TensorDataset(val_dataset,val_labels_torch)

In [10]:
train_dataset.shape,train_labels_torch.shape

(torch.Size([8177, 3, 224, 224]), torch.Size([8177, 1]))

In [11]:
val_dataset_torch[0]

(tensor([[[0.2980, 0.2667, 0.1569,  ..., 0.2510, 0.0471, 0.0353],
          [0.2275, 0.1490, 0.2157,  ..., 0.1020, 0.0196, 0.0078],
          [0.2000, 0.1098, 0.2745,  ..., 0.0510, 0.0275, 0.0157],
          ...,
          [0.1176, 0.2118, 0.1843,  ..., 0.1176, 0.1373, 0.1451],
          [0.1490, 0.1843, 0.1804,  ..., 0.1765, 0.1569, 0.1451],
          [0.2431, 0.2784, 0.2118,  ..., 0.1529, 0.1216, 0.1059]],
 
         [[0.3765, 0.3373, 0.2275,  ..., 0.2863, 0.0549, 0.0314],
          [0.3137, 0.2314, 0.2980,  ..., 0.1373, 0.0275, 0.0039],
          [0.2941, 0.2039, 0.3647,  ..., 0.0824, 0.0392, 0.0157],
          ...,
          [0.1176, 0.2118, 0.1882,  ..., 0.1333, 0.1529, 0.1608],
          [0.1490, 0.1843, 0.1843,  ..., 0.1843, 0.1647, 0.1529],
          [0.2431, 0.2784, 0.2157,  ..., 0.1569, 0.1255, 0.1059]],
 
         [[0.3765, 0.3608, 0.2588,  ..., 0.2235, 0.0431, 0.0510],
          [0.2667, 0.2000, 0.2706,  ..., 0.0745, 0.0157, 0.0235],
          [0.1961, 0.1176, 0.2902,  ...,

In [12]:
from torchvision.models import resnet101, ResNet101_Weights
import torch.nn as nn
import torch.optim as optim

resnet = resnet101(weights=ResNet101_Weights.IMAGENET1K_V1)
for param in resnet.parameters():
    param.requires_grad = False


resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 256),  # From ResNet's output to 256 units
    nn.ReLU(),                              # Activation function
    nn.BatchNorm1d(256),                    # Normalization for stability
    nn.Linear(256, 256),                    # Additional dense layer
    nn.ReLU(),                              # Activation again
    nn.Dropout(0.3),                        # Dropout to reduce overfitting
    nn.BatchNorm1d(256),                    # Another batch norm layer
    nn.Linear(256, 120)                     # Final layer: 120 output classes
)
model_torch = resnet

optimizier=optim.Adam(model_torch.parameters(),lr=1e-4)
loss_fn=nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 139MB/s]


In [13]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset_torch, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset_torch, batch_size=64, shuffle=False)

n_epochs=30
model_torch=model_torch.to('cuda')
for i in range(1,n_epochs+1):
    model_torch.train()
    training_loss=0.0
    for img,label in train_loader:
        img=img.to('cuda')
        label=label.to('cuda')
        label=label.squeeze(1)
        outputs=model_torch(img)
        optimizier.zero_grad()
        losses=loss_fn(outputs,label)
        training_loss+=losses.item()
        losses.backward()
        optimizier.step()

    model_torch.eval()
    val_loss=0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for img,label in val_loader:
            img=img.to('cuda')
            label=label.to('cuda')
            label=label.squeeze(1)
            outputs=model_torch(img)
            loss=loss_fn(outputs,label)
            val_loss+=loss.item()

            _, predicted = torch.max(outputs, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print(f"Epoch [{i}/{n_epochs}], Training Loss: {training_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")


Epoch [1/30], Training Loss: 3.9094, Validation Loss: 3.0089, Accuracy: 59.56%
Epoch [2/30], Training Loss: 2.6803, Validation Loss: 2.2478, Accuracy: 72.03%
Epoch [3/30], Training Loss: 2.0721, Validation Loss: 1.7863, Accuracy: 75.70%
Epoch [4/30], Training Loss: 1.6680, Validation Loss: 1.4940, Accuracy: 78.78%
Epoch [5/30], Training Loss: 1.3946, Validation Loss: 1.3100, Accuracy: 79.85%
Epoch [6/30], Training Loss: 1.1907, Validation Loss: 1.1677, Accuracy: 81.12%
Epoch [7/30], Training Loss: 1.0418, Validation Loss: 1.0359, Accuracy: 80.64%
Epoch [8/30], Training Loss: 0.9160, Validation Loss: 0.9667, Accuracy: 80.93%
Epoch [9/30], Training Loss: 0.8262, Validation Loss: 0.9083, Accuracy: 81.61%
Epoch [10/30], Training Loss: 0.7462, Validation Loss: 0.8657, Accuracy: 82.15%
Epoch [11/30], Training Loss: 0.6841, Validation Loss: 0.8140, Accuracy: 81.56%
Epoch [12/30], Training Loss: 0.6137, Validation Loss: 0.7960, Accuracy: 81.81%
Epoch [13/30], Training Loss: 0.5651, Validation 

In [1]:
torch.save(resnet.state_dict(), 'resnet_custom_head.pth')


NameError: name 'torch' is not defined

In [2]:
os.listdir('/content/dog-breed-identification/test')[1:10]

NameError: name 'os' is not defined

In [ ]:

test=os.listdir('/content/dog-breed-identification/test')
test_dataset=create_dataset(test,'/content/dog-breed-identification/test',val_transform)
test_dataset=test_dataset.to(device='cuda')

In [ ]:
import pandas as pd
output=pd.DataFrame(columns=['file',*breeds.keys()])

In [ ]:
output

In [ ]:
breeds

In [ ]:
import torch.nn.functional as F
i=0
for file,sample in zip(os.listdir('/content/dog-breed-identification/test'),test_dataset):
    sample=sample.cuda()
    output_values=model_torch(sample.unsqueeze(dim=0))
    output_values=output_values.cpu()
    probabilities = F.softmax(output_values, dim=1)
    output.loc[i]=[file,*probabilities[0].detach().numpy()]
    i+=1

In [ ]:
output.head()

In [ ]:
output.to_csv('test_submission.csv',index=False)

In [ ]:
torch.save(model.module.state_dict() if hasattr(model, "module") else model.state_dict(), "dog_breed_model.pth"); from google.colab import drive; drive.mount('/content/drive', force_remount=True); __import__('shutil').copy("dog_breed_model.pth", "/content/drive/MyDrive/dog_breed_model.pth")


In [5]:
!cp /content/resnet_custom_head.pth /content/drive/MyDrive/

In [8]:
import torch
import torchvision.models as models

# 1. Recreate the correct model architecture: ResNet-50
resnet = models.resnet50(weights=None)  # Use weights=None instead of deprecated pretrained=False

# 2. Replace the final classification layer with the correct number of output classes
# 👇 Change 120 to however many classes your model was trained on
resnet.fc = torch.nn.Linear(resnet.fc.in_features, 120)

# 3. Load model weights
model_path = "/content/resnet_custom_head.pth"  # Your saved model path
resnet.load_state_dict(torch.load(model_path, map_location="cpu"))
resnet.eval()

# 4. Print number of output classes
print(f"✅ Number of output classes: {resnet.fc.out_features}")


RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "layer3.6.conv1.weight", "layer3.6.bn1.weight", "layer3.6.bn1.bias", "layer3.6.bn1.running_mean", "layer3.6.bn1.running_var", "layer3.6.bn1.num_batches_tracked", "layer3.6.conv2.weight", "layer3.6.bn2.weight", "layer3.6.bn2.bias", "layer3.6.bn2.running_mean", "layer3.6.bn2.running_var", "layer3.6.bn2.num_batches_tracked", "layer3.6.conv3.weight", "layer3.6.bn3.weight", "layer3.6.bn3.bias", "layer3.6.bn3.running_mean", "layer3.6.bn3.running_var", "layer3.6.bn3.num_batches_tracked", "layer3.7.conv1.weight", "layer3.7.bn1.weight", "layer3.7.bn1.bias", "layer3.7.bn1.running_mean", "layer3.7.bn1.running_var", "layer3.7.bn1.num_batches_tracked", "layer3.7.conv2.weight", "layer3.7.bn2.weight", "layer3.7.bn2.bias", "layer3.7.bn2.running_mean", "layer3.7.bn2.running_var", "layer3.7.bn2.num_batches_tracked", "layer3.7.conv3.weight", "layer3.7.bn3.weig

In [9]:
import torch
from torchvision import models

# 1) Load raw state_dict
ckpt = torch.load('/content/resnet_custom_head.pth', map_location='cpu')

# 2) Recreate ResNet-101 (no pretrained weights)
resnet = models.resnet101(weights=None)


In [10]:
num_classes = 120
resnet.fc = torch.nn.Linear(resnet.fc.in_features, num_classes)


In [14]:
import torch

# 1) Load your checkpoint
ckpt = torch.load('/content/resnet_custom_head.pth', map_location='cpu')

# 2) Find all fc-layer weight tensors (they are 2D: [out_features, in_features])
fc_weights = [(k, v) for k, v in ckpt.items() if k.startswith('fc') and v.ndim == 2]

# 3) Take the *last* one (the final Linear in your head) and read its out_features
last_fc_key, last_fc_tensor = sorted(fc_weights, key=lambda x: x[0])[-1]
num_classes = last_fc_tensor.shape[0]

print(f"🔍 The final fc-layer is “{last_fc_key}” with out_features = {num_classes}")

🔍 The final fc-layer is “fc.7.weight” with out_features = 120


In [13]:
import torch

# 1) Load your checkpoint
ckpt = torch.load('/content/resnet_custom_head.pth', map_location='cpu')

# 2) Find all fc-layer weight tensors (they are 2D: [out_features, in_features])
fc_weights = [(k, v) for k, v in ckpt.items() if k.startswith('fc') and v.ndim == 2]

# 3) Take the *last* one (the final Linear in your head) and read its out_features
last_fc_key, last_fc_tensor = sorted(fc_weights, key=lambda x: x[0])[-1]
num_classes = last_fc_tensor.shape[0]

print(f"🔍 The final fc-layer is “{last_fc_key}” with out_features = {num_classes}")

🔍 The final fc-layer is “fc.7.weight” with out_features = 120
